- Fiance api: https://pypi.org/project/yfinance/
- Earnings calendar for next day: https://www.earningswhispers.com/calendar?sb=p&d=1&t=all
- More Click: https://www.earningswhispers.com/morecalendar?sb=p&d=1&v=s
- Raportowanie z danych z yfinance

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import re
from bs4 import BeautifulSoup

html_tags_regex = r'<[^<]+?>'

def parse_tickers(ticker_divs):
    return [re.sub(html_tags_regex, '', str(x)) for x in ticker_divs]

In [21]:
URL = "https://www.earningswhispers.com/calendar?sb=p&d=1&t=all"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [22]:
ticker_elements = soup.find_all("div", class_="ticker")
tickers = parse_tickers(ticker_elements)
tickers

['TLRY', 'CMC', 'AZZ', 'ACCD', 'LMNR']

In [23]:
URL = "https://www.earningswhispers.com/morecalendar?sb=p&d=1&v=s"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [24]:
ticker_elements = soup.find_all("div", class_="ticker")
tickers.extend(parse_tickers(ticker_elements))
tickers

['TLRY', 'CMC', 'AZZ', 'ACCD', 'LMNR', 'VOXX']

In [36]:
import yfinance as yf

def get_financial_data(ticker):

    stock = yf.Ticker(ticker)

    #info 
    info = stock.info
    
    # get historical market data
    hist = stock.history(period="3mo", interval="1d")

    quarterly_financials = stock.quarterly_financials

    # show major holders
    major_holders = stock.major_holders

    # show news
    news = stock.news

    # get option chain for specific expiration
    return info, hist, quarterly_financials, major_holders, news
    # data available via: opt.calls, opt.puts

In [39]:
AAPL = get_financial_data("AAPL")

In [42]:
AAPL[0]

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 154000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience wit

In [188]:
import pickle
import datetime

today = datetime.datetime.today()
tomorrow = today + datetime.timedelta(days=1)
tomorrow_date = str(tomorrow).split(" ")[0]

for ticker in tickers:
    filename = f'tmp/{ticker}_{tomorrow_date}.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(get_financial_data(ticker), handle, protocol=pickle.HIGHEST_PROTOCOL)



In [189]:
decoded_tickers = dict()
for ticker in tickers:
    filename = f'tmp/{ticker}_{tomorrow_date}.pickle'
    print(f'For {ticker}:')
    with open(filename, 'rb') as handle:
        decoded_tickers[ticker] = pickle.load(handle)

For TLRY:
For CMC:
For AZZ:
For ACCD:
For LMNR:
For VOXX:


# Report Generation

In [ ]:
from fpdf import FPDF
import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

today = datetime.datetime.today()
tomorrow = today + datetime.timedelta(days=1)
tomorrow_date = str(tomorrow).split(" ")[0]


pdf = FPDF()
  

# Add a page
pdf.add_page()
  
# set style and size of font 
# that you want in the pdf
pdf.set_font("Arial", size = 15)
# create a cell
pdf.cell(200, 10, txt = "Stock Earnings Report", 
         ln = 1, align = 'C')

pdf.cell(200, 10, txt = tomorrow_date, 
         ln = 1, align = 'C')


for ticker in tickers:
    filename = f'tmp/{ticker}_{tomorrow_date}.pickle'
    print(f'For {ticker}:')
    with open(filename, 'rb') as handle:
        decoded_tickers[ticker] = pickle.load(handle)

        pdf.set_font("Arial", size = 15)
        pdf.cell(150, 8, txt = ticker,
                 ln = 1, align = 'A')
        pdf.set_font("Arial", size = 10)
        sector = decoded_tickers[ticker][0]['sector']
        employees = decoded_tickers[ticker][0]['fullTimeEmployees']
        summary = decoded_tickers[ticker][0]['longBusinessSummary']
        website = decoded_tickers[ticker][0]['website']
        pdf.cell(150, 4, txt = f'Sector: {sector}',
                 ln = 0, align = 'A')
        pdf.cell(150, 4, txt = f'Employees: {employees}',
                 ln = 1, align = 'B')
        pdf.set_font("Arial", size = 8)
        pdf.cell(150, 4, txt = f'Website: {website}',link=website,
                 ln = 1, align = 'A')
        pdf.multi_cell(180, 3, txt = summary, align = 'A')
        history = decoded_tickers[ticker][1]
        # Volume fig
        pdf.set_font("Arial", size = 10)
        pdf.cell(150, 4, txt = f'Volume', ln = 1, align = 'A')
        fig = plt.figure()
        fig = history['Volume'].plot().get_figure()
        fig.savefig(f'tmp/volume_{ticker}.png',dpi=300)
        pdf.image(f'tmp/volume_{ticker}.png',w=100)
        
        # Price
        pdf.set_font("Arial", size = 10)
        pdf.cell(150, 4, txt = f'Price', ln = 1, align = 'A')
        fig = plt.figure()
        fig = history.loc[:,['Open','Close']].plot().get_figure()
        fig.savefig(f'tmp/price_{ticker}.png',dpi=300)
        pdf.image(f'tmp/price_{ticker}.png',w=100)

        pdf.add_page()
pdf.output(f'Report_{tomorrow_date}.pdf') 

For TLRY:
For CMC:
For AZZ:
For ACCD:
For LMNR:
For VOXX:
